In [1]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig

C:\Users\latha\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
ws = Workspace.from_config()
print('Workspace', ws)

Workspace Workspace.create(name='MachineLearningWorkspace', subscription_id='73bd14a5-edfe-4e77-a22d-8de4e1ca1301', resource_group='AzureMachineLearning')


In [3]:
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_name = 'ClassificationInstance'

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Compute Instance already exits: ', compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDATD_D2_V2', max_nodes=2)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
    print('Compute Instance Created')

Compute Instance already exits:  ClassificationInstance


In [7]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
print('DataStore: \n', datastore)
datastore.upload(src_dir='Datafolder/', target_path='Data/')
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'Data/diabetes.csv'))
dataset = dataset.register(workspace=ws, name='Diabetic_Data', description='Diabetic_Data', create_new_version=True)
print('Dataset Registered', dataset.name)

DataStore: 
 {
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-02cb5d08-87c7-4ca3-a008-ab4f93ceac63",
  "account_name": "machinelearnin2333951653",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
Uploading an estimated of 1 files
Target already exists. Skipping upload for Data/diabetes.csv
Uploaded 0 files
Dataset Registered Diabetic_Data


In [8]:
experiment_name = 'experiment'
exp = Experiment(workspace=ws, name=experiment_name)

In [10]:
env = Environment.from_conda_specification(name='azure_ml', file_path='envfile.yml')
config = ScriptRunConfig(source_directory='.',script='training.py', compute_target=compute_target, environment=env)
execution = exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: experiment_1725258838_1454b874
Web View: https://ml.azure.com/runs/experiment_1725258838_1454b874?wsid=/subscriptions/73bd14a5-edfe-4e77-a22d-8de4e1ca1301/resourcegroups/AzureMachineLearning/workspaces/MachineLearningWorkspace&tid=4dd0e38f-dff7-492a-9237-02dbb50cea63

Streaming user_logs/std_log.txt

<function accuracy_score at 0x148972c2b8b0>
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.037288665771484375 seconds

Execution Summary
RunId: experiment_1725258838_1454b874
Web View: https://ml.azure.com/runs/experiment_1725258838_1454b874?wsid=/subscriptions/73bd14a5-edfe-4e77-a22d-8de4e1ca1301/resourcegroups/AzureMachineLearning/workspaces/MachineLearningWorkspace&tid=4dd0e38f-dff7-492a-9237-02dbb50cea63



{'runId': 'experiment_1725258838_1454b874',
 'target': 'ClassificationInstance',
 'status': 'Completed',
 'startTimeUtc': '2024-09-02T06:34:15.816886Z',
 'endTimeUtc': '2024-09-02T06:34:47.949861Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcdsi',
  '_azureml.ClusterName': 'ClassificationInstance',
  'ContentSnapshotId': '011fe4f7-f9c6-471b-9d29-395ece9fee57',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'ClassificationInstance',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough'

In [11]:
from azureml.core.model import Model

model_path = 'Logisticmodel.joblib'
model_name = 'Logisticmodel'
model_registered = Model.register(model_name=model_name, model_path=model_path, workspace=ws)


Registering model Logisticmodel


In [12]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

conda_env = Environment('my-environment')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [13]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, webservice

infer_config = InferenceConfig(entry_script='score.py', environment=conda_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [16]:
service = Model.deploy(name='logistic-model-deployment',
                      models=[model_registered], 
                      inference_config=infer_config,
                      deployment_config=aci_config,
                      workspace=ws)
service.wait_for_deployment(show_output=True)

C:\Users\latha\AppData\Local\Temp\ipykernel_4784\1857234899.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(name='logistic-model-deployment',


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-02 12:48:33+05:30 Creating Container Registry if not exists.
2024-09-02 12:48:33+05:30 Registering the environment.
2024-09-02 12:48:40+05:30 Use the existing image.
2024-09-02 12:48:42+05:30 Submitting deployment to compute.
2024-09-02 12:48:46+05:30 Checking the status of deployment logistic-model-deployment..
2024-09-02 12:50:02+05:30 Checking the status of inference endpoint logistic-model-deployment.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [17]:
scoring_uri = service.scoring_uri
scoring_uri

'http://2d521a71-a325-41f1-a824-f3fc076b511c.eastus2.azurecontainer.io/score'